In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('chorales/train/chorale_000.csv')

In [3]:
df

,note0,note1,note2,note3
0,74,70,65,58
1,74,70,65,58
2,74,70,65,58
3,74,70,65,58
4,75,70,58,55
...,...,...,...,...
187,70,65,62,46
188,70,65,62,46
189,70,65,62,46
190,70,65,62,46


In [4]:
import os

train_files=sorted([os.path.join('chorales','train',f) for f in os.listdir(os.path.join('chorales','train'))if f.endswith('.csv')])
test_files=sorted([os.path.join('chorales','test',f) for f in os.listdir(os.path.join('chorales','test'))if f.endswith('.csv')])
valid_files=sorted([os.path.join('chorales','valid',f) for f in os.listdir(os.path.join('chorales','valid'))if f.endswith('.csv')])

In [6]:
train_data=[pd.read_csv(f).values.tolist() for f in train_files]
test_data=[pd.read_csv(f).values.tolist() for f in test_files]
valid_data=[pd.read_csv(f).values.tolist() for f in valid_files]